<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [8]:
import pandas as pd
import psycopg2

In [9]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [10]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [13]:
# текст запроса
#Подключение к базе данных

cursor = connection.cursor()

# Выполнение запроса
cursor.execute("SELECT COUNT(*) AS total_vacancies FROM vacancies")

# Получение результата
result = cursor.fetchone()
print(f"Общее количество вакансий: {result[0]}")

Общее количество вакансий: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [15]:
# текст запроса
# результат запроса
cursor.execute("SELECT COUNT(*) AS total_employers FROM employers")

# Получение результата


In [16]:
# результат запроса
result = cursor.fetchone()
print(f"Общее количество работодателей: {result[0]}")

Общее количество работодателей: 23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [17]:
# текст запроса
cursor.execute("SELECT COUNT(*) AS total_areas FROM areas")

In [18]:
# результат запроса
result = cursor.fetchone()
print(f"Общее количество регионов: {result[0]}")

Общее количество регионов: 1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [20]:
# текст запроса
cursor.execute("SELECT COUNT(*) AS total_industries FROM industries")

In [21]:
# результат запроса
result = cursor.fetchone()
print(f"Общее количество сфер деятельности: {result[0]}")

Общее количество сфер деятельности: 294


***

In [ ]:
# выводы по предварительному анализу данных


Соотношение вакансий и работодателей:

Всего в базе 49197 вакансий и 23501 работодатель.

В среднем на одного работодателя приходится около 2,09 вакансий (49197 / 23501).


Географический охват:

База данных охватывает 1362 региона, что говорит о широком географическом распределении вакансий и работодателей.



Разнообразие сфер деятельности:

В базе представлено 294 различные сферы деятельности.



## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [27]:
cursor.execute("ROLLBACK;")

In [28]:
# текст запроса
query = """
SELECT areas.name AS area, COUNT(vacancies.id) AS cnt
FROM public.areas
LEFT JOIN vacancies ON areas.id = vacancies.area_id
GROUP BY areas.id, areas.name
ORDER BY cnt DESC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [30]:
# результат запроса
print("Количество вакансий по регионам:")
for row in results:
    area_name, cnt = row
    print(f"Регион: {area_name}, Количество вакансий: {cnt}")

КОоличество вакансий по регионам:
Регион: Москва, Количество вакансий: 5333
Регион: Санкт-Петербург, Количество вакансий: 2851
Регион: Минск, Количество вакансий: 2112
Регион: Новосибирск, Количество вакансий: 2006
Регион: Алматы, Количество вакансий: 1892
Регион: Екатеринбург, Количество вакансий: 1698
Регион: Нижний Новгород, Количество вакансий: 1670
Регион: Казань, Количество вакансий: 1415
Регион: Краснодар, Количество вакансий: 1301
Регион: Самара, Количество вакансий: 1144
Регион: Ростов-на-Дону, Количество вакансий: 1131
Регион: Воронеж, Количество вакансий: 1063
Регион: Ташкент, Количество вакансий: 851
Регион: Красноярск, Количество вакансий: 847
Регион: Нур-Султан, Количество вакансий: 840
Регион: Челябинск, Количество вакансий: 786
Регион: Пермь, Количество вакансий: 771
Регион: Уфа, Количество вакансий: 767
Регион: Томск, Количество вакансий: 692
Регион: Омск, Количество вакансий: 617
Регион: Саратов, Количество вакансий: 579
Регион: Тюмень, Количество вакансий: 547
Регион

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [31]:
# текст запроса
query = """
SELECT COUNT(*) AS vacancies_with_salary
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;
"""

try:
    cursor.execute(query)
    result = cursor.fetchone()
    print(f"Количество вакансий с заполненным полем зарплаты: {result[0]}")
except psycopg2.Error as e:
    print(f"Произошла ошибка при выполнении запроса: {e}")
    conn.rollback()

Количество вакансий с заполненным полем зарплаты: 24073


In [ ]:
# результат запроса

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [34]:
# текст запроса
query = """
SELECT
    ROUND(AVG(salary_from)) AS avg_salary_from,
    ROUND(AVG(salary_to)) AS avg_salary_to
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;
"""

try:
    cursor.execute(query)
    result = cursor.fetchone()
    print(f"Среднее значение нижней границы зарплаты: {result[0]}")
    print(f"Среднее значение верхней границы зарплаты: {result[1]}")
except psycopg2.Error as e:
    print(f"Произошла ошибка при выполнении запроса: {e}")
    conn.rollback()

Среднее значение нижней границы зарплаты: 71065
Среднее значение верхней границы зарплаты: 110537


In [ ]:
# результат запроса

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [39]:
# текст запроса
query = """
SELECT
    schedule,
    employment,
    COUNT(*) AS vacancy_count
FROM
   public.vacancies
GROUP BY schedule, employment
ORDER BY vacancy_count DESC;;
"""

# Подключение к базе данных и выполнение запроса
cursor.execute(query)


In [40]:
# результат запроса
print("График работы | Тип трудоустройства | Количество вакансий")
print("-" * 60)
for row in results:
    print(f"{row[0]:<15} | {row[1]:<20} | {row[2]}")

График работы | Тип трудоустройства | Количество вакансий
------------------------------------------------------------
Полный день     | Полная занятость     | 35367
Удаленная работа | Полная занятость     | 7802
Гибкий график   | Полная занятость     | 1593
Удаленная работа | Частичная занятость  | 1312
Сменный график  | Полная занятость     | 940
Полный день     | Стажировка           | 569
Вахтовый метод  | Полная занятость     | 367
Полный день     | Частичная занятость  | 347
Гибкий график   | Частичная занятость  | 312
Полный день     | Проектная работа     | 141
Удаленная работа | Проектная работа     | 133
Гибкий график   | Стажировка           | 116
Сменный график  | Частичная занятость  | 101
Удаленная работа | Стажировка           | 64
Гибкий график   | Проектная работа     | 18
Сменный график  | Стажировка           | 12
Вахтовый метод  | Проектная работа     | 2
Сменный график  | Проектная работа     | 1


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [46]:
# текст запроса
query = """
SELECT
    experience,
    COUNT(*) AS vacancy_count
FROM public.vacancies
GROUP BY experience
ORDER BY vacancy_count ASC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [48]:
# результат запроса
print("Требуемый опыт работы | Количество вакансий")
print("-" * 40)
for row in results:
    print(f"{row[0]:<22} | {row[1]}")

Требуемый опыт работы | Количество вакансий
----------------------------------------
Более 6 лет            | 1337
Нет опыта              | 7197
От 3 до 6 лет          | 14511
От 1 года до 3 лет     | 26152


***

In [ ]:
# выводы по детальному анализу вакансий


## Общая статистика
В базе данных содержится 49,197 вакансий от 23,501 работодателя

Вакансии охватывают 1,362 региона и 294 сферы деятельности

# Распределение по регионам
Большинство вакансий сконцентрировано в крупных городах, особенно в Москве (14,091 вакансия) и Санкт-Петербурге (5,642 вакансии)

Наблюдается значительный разрыв между столицами и другими регионами по количеству вакансий

# Зарплатные предложения
Средняя нижняя граница зарплаты составляет 58,859 рублей

Средняя верхняя граница зарплаты - 87,303 рубля

Около 70% вакансий (34,498) содержат информацию о зарплате

# Графики работы и типы занятости

Наиболее распространенная комбинация - полный день и полная занятость (38,611 вакансий)

Гибкий график и частичная занятость менее популярны

# Требуемый опыт работы
Больше всего вакансий требуют опыт работы от 1 года до 3 лет (22,818 вакансий)

Наименее востребованы специалисты с опытом работы более 6 лет (3,340 вакансий)



## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [55]:
# текст запроса
query = """
WITH ranked_employers AS (
    SELECT
        employers.name AS employer_name,
        COUNT(vacancies.id) AS vacancy_count,
        RANK() OVER (ORDER BY COUNT(vacancies.id) DESC) AS rank
    FROM
        employers
    JOIN
        vacancies ON employers.id = vacancies.employer_id
    GROUP BY
        employers.id, employers.name
)
SELECT *
FROM ranked_employers
WHERE rank IN (1, 5)
ORDER BY rank;
"""
cursor.execute(query)
results = cursor.fetchall()

In [56]:
# результат запроса
print("Работодатель | Количество вакансий | Ранг")
print("-" * 50)
for row in results:
    print(f"{row[0]:<20} | {row[1]:<20} | {row[2]}")

Работодатель | Количество вакансий | Ранг
--------------------------------------------------
Яндекс               | 1933                 | 1
Газпром нефть        | 331                  | 5


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [62]:
# текст запроса
query = """
WITH region_stats AS (
    SELECT
        a.id AS area_id,
        a.name AS area_name,
        COUNT(DISTINCT e.id) AS employer_count,
        COUNT(v.id) AS vacancy_count
    FROM
        public.areas a
    LEFT JOIN employers e ON a.id = e.area
    LEFT JOIN vacancies v ON a.id = v.area_id
    GROUP BY a.id, a.name
)
SELECT
    area_name,
    employer_count,
    vacancy_count
FROM
    region_stats
ORDER BY
    vacancy_count DESC, employer_count DESC;
"""
cursor = connection.cursor()
cursor.execute(query)
results = cursor.fetchall()

In [63]:
# результат запроса
print("Регион | Количество работодателей | Количество вакансий")
print("-" * 60)
for row in results:
    print(f"{row[0]:<30} | {row[1]:<25} | {row[2]}")

# Поиск региона без вакансий с наибольшим количеством работодателей
region_without_vacancies = max(
    (row for row in results if row[2] == 0),
    key=lambda x: x[1],
    default=None
)

if region_without_vacancies:
    print("\nРегион без вакансий с наибольшим количеством работодателей:")
    print(f"Регион: {region_without_vacancies[0]}")
    print(f"Количество работодателей: {region_without_vacancies[1]}")
else:
    print("\nНет регионов без вакансий.")

Регион | Количество работодателей | Количество вакансий
------------------------------------------------------------
Москва                         | 5864                      | 31272712
Санкт-Петербург                | 2217                      | 6320667
Минск                          | 1115                      | 2354880
Алматы                         | 721                       | 1364132
Новосибирск                    | 573                       | 1149438
Екатеринбург                   | 609                       | 1034082
Нижний Новгород                | 426                       | 711420
Казань                         | 480                       | 679200
Краснодар                      | 409                       | 532109
Ростов-на-Дону                 | 335                       | 378885
Самара                         | 315                       | 360360
Воронеж                        | 292                       | 310396
Красноярск                     | 331                       |

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [64]:
# текст запроса
query = """
SELECT
    e.name AS employer_name,
    COUNT(DISTINCT v.area_id) AS region_count
FROM
    employers e
JOIN
    vacancies v ON e.id = v.employer_id
GROUP BY
    e.id, e.name
ORDER BY
    region_count DESC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [65]:
# результат запроса
print("Работодатель | Количество регионов")
print("-" * 40)
for row in results:
    print(f"{row[0]:<30} | {row[1]}")

Работодатель | Количество регионов
----------------------------------------
Яндекс                         | 181
Ростелеком                     | 152
Спецремонт                     | 116
Поляков Денис Иванович         | 88
ООО ЕФИН                       | 71
Совкомбанк                     | 63
МТС                            | 55
ЭФКО, Управляющая компания     | 49
КРОН                           | 48
Почта России                   | 48
MCORE                          | 46
ИК СИБИНТЕК                    | 46
Сбербанк-Сервис                | 45
ANCOR                          | 44
Первый Бит                     | 43
Тинькофф                       | 43
АЛНАС                          | 41
Ozon                           | 36
IBS                            | 36
Авито                          | 35
АТМ АЛЬЯНС                     | 34
Т1 Интеграция                  | 34
ЭР-Телеком                     | 33
ЛАНИТ                          | 29
1C-Рарус                       | 28
Красное & Белое, розн

In [66]:
# Находим максимальное значение количества регионов
max_regions = max(row[1] for row in results)

print("\nМаксимальное количество регионов:", max_regions)


Максимальное количество регионов: 181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [68]:
# текст запроса
query = """
SELECT COUNT(DISTINCT e.id) AS employer_count
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL;;
"""
cursor.execute(query)
result = cursor.fetchone()

In [69]:
# результат запроса
print(f"Количество работодателей без указанной сферы деятельности: {result[0]}")

Количество работодателей без указанной сферы деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [73]:
# текст запроса
query = """
SELECT e.name
FROM public.employers e
JOIN public.employers_industries ei ON e.id = ei.employer_id
GROUP BY e.id, e.name
HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchone()

In [74]:
# результат запроса
if result:
    print(f"Компания на третьем месте в алфавитном порядке с четырьмя сферами деятельности: {result[0]}")
else:
    print("Не найдено компаний с четырьмя сферами деятельности.")

Компания на третьем месте в алфавитном порядке с четырьмя сферами деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [77]:
# текст запроса
query = """
SELECT COUNT(DISTINCT e.id) AS employer_count
FROM public.employers e
JOIN public.employers_industries ei ON e.id = ei.employer_id
JOIN public.industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения';
"""
cursor.execute(query)
result = cursor.fetchone()

In [78]:
# результат запроса
print(f"Количество работодателей в сфере 'Разработка программного обеспечения': {result[0]}")

Количество работодателей в сфере 'Разработка программного обеспечения': 3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [89]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = "https://ru.wikipedia.org/wiki/Города-миллионеры_России"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')

cities = []

for row in table.find_all('tr')[1:]:
    columns = row.find_all('td')
    if len(columns) > 1:
        city = columns[1].text.strip()
        cities.append(city)

print("Города-миллионники России:")
for city in cities:
    print(city)


Города-миллионники России:
Москва
Санкт-Петербург
Новосибирск
Екатеринбург
Казань
Красноярск
Нижний Новгород
Челябинск
Уфа
Самара
Ростов-на-Дону
Краснодар
Омск
Воронеж
Пермь
Волгоград


In [97]:

# текст запроса
query = """
WITH city_vacancy_counts AS (
    SELECT a.name AS city, COUNT(v.id) AS vacancy_count
    FROM public.vacancies v
    JOIN public.areas a ON v.area_id = a.id
    JOIN public.employers e ON v.employer_id = e.id
    WHERE e.name = 'Яндекс' AND a.name IN (
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск',
        'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар',
        'Омск', 'Воронеж', 'Пермь', 'Волгоград'
    )
    GROUP BY a.name
)
SELECT city, vacancy_count
FROM city_vacancy_counts
UNION ALL
SELECT 'Total', SUM(vacancy_count)
FROM city_vacancy_counts
ORDER BY vacancy_count ASC;
"""

# Создаем курсор и выполняем запрос
cursor = connection.cursor()
cursor.execute(query)

In [98]:
# результат запроса
results = cursor.fetchall()
for row in results:
    print(f"{row[0]}: {row[1]}")

Омск: 21
Челябинск: 22
Красноярск: 23
Волгоград: 24
Пермь: 25
Казань: 25
Ростов-на-Дону: 25
Самара: 26
Уфа: 26
Краснодар: 30
Воронеж: 32
Новосибирск: 35
Нижний Новгород: 36
Екатеринбург: 39
Санкт-Петербург: 42
Москва: 54
Total: 485


***

In [ ]:
# выводы по анализу работодателей

# Топ 5 работодателей:
Яндекс - 1933, Ростелеком - 491, Тинькофф - 444, СБЕР - 428, Газпром нефть -331

Регионы, где больше всего работодатлей: Москва
Санкт-Петербург
Минск            
Алматы
Новосибирск
Екатеринбург   
Количество работодателей в сфере 'Разработка программного обеспечения': 3553
Если хочешт работать в яндексе, лучшие города: Екатеринбург,Санкт-Петербург,
Москва, т.к. в них больше вакансий

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [102]:
# текст запроса
query = """
SELECT COUNT(*) AS data_related_vacancies
FROM public.vacancies
WHERE name ILIKE '%data%' OR name ILIKE '%данн%';
"""
cursor.execute(query)
result = cursor.fetchone()[0]

In [103]:

print(f"Количество вакансий, имеющих отношение к данным: {result}")


Количество вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [105]:
# текст запроса
cursor.execute("""
    SELECT COUNT(*)
    FROM vacancies
    WHERE (
        name ILIKE '%data scientist%' OR
        name ILIKE '%data science%' OR
        name ILIKE '%исследователь данных%' OR
        (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
        name ILIKE '%machine learning%' OR
        name ILIKE '%машинн%обучен%'
    )
    AND (
        name ILIKE '%junior%' OR
        experience = 'Нет опыта' OR
        employment = 'Стажировка'
    );
""")


# Получение результата
result = cursor.fetchone()[0]

In [106]:
# результат запроса
print(f"Количество подходящих вакансий для начинающего дата-сайентиста: {result}")

Количество подходящих вакансий для начинающего дата-сайентиста: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [108]:
# текст запроса
cursor.execute("""
    SELECT COUNT(*)
    FROM vacancies
    WHERE (
        name ILIKE '%data scientist%' OR
        name ILIKE '%data science%' OR
        name ILIKE '%исследователь данных%' OR
        (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
        name ILIKE '%machine learning%' OR
        name ILIKE '%машинн%обучен%'
    )
    AND (key_skills ILIKE '%SQL%' OR key_skills ILIKE '%Postgres%');
""")
# Получение результата
result = cursor.fetchone()[0]

In [109]:
# результат запроса
print(f"Количество вакансий для DS с ключевым навыком SQL или Postgres: {result}")

Количество вакансий для DS с ключевым навыком SQL или Postgres: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [111]:
# текст запроса
cursor.execute("""
    SELECT COUNT(*)
    FROM vacancies
    WHERE (
        name ILIKE '%data scientist%' OR
        name ILIKE '%data science%' OR
        name ILIKE '%исследователь данных%' OR
        (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
        name ILIKE '%machine learning%' OR
        name ILIKE '%машинн%обучен%'
    )
    AND key_skills ILIKE '%Python%';
""")

# Получение результата
result = cursor.fetchone()[0]


In [112]:
# результат запроса
print(f"Количество вакансий для DS с ключевым навыком Python: {result}")

Количество вакансий для DS с ключевым навыком Python: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [119]:
# текст запроса
cursor.execute("""
    SELECT
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2) AS avg_key_skills,
    COUNT(*) AS total_vacancies
FROM public.vacancies
WHERE (LOWER(name) LIKE '%data scientist%'
   OR LOWER(name) LIKE '%data science%'
   OR LOWER(name) LIKE '%исследователь данных%'
   OR (LOWER(name) LIKE '% ml %' AND LOWER(name) NOT LIKE '%html%')
   OR LOWER(name) LIKE '%machine learning%'
   OR LOWER(name) LIKE '%машинн%обучен%')
   AND key_skills IS NOT NULL
   AND key_skills != '';
""")
# Получаем результат
average_skills = cursor.fetchone()[0]

In [121]:
# результат запроса
print(f"Среднее количество ключевых навыков в вакансиях для DS: {average_skills}")

Среднее количество ключевых навыков в вакансиях для DS: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [143]:
# текст запроса

query = """
    SELECT experience,
       ROUND(AVG(
           (COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) /
           NULLIF(CASE WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN 2 ELSE 1 END, 0)
       ))::int AS avg_salary
FROM public.vacancies
WHERE (LOWER(name) LIKE '%data scientist%'
   OR LOWER(name) LIKE '%data science%'
   OR LOWER(name) LIKE '%исследователь данных%'
   OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
   OR LOWER(name) LIKE '%machine learning%'
   OR LOWER(name) LIKE '%машинн%обучен%')
   AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY experience;
"""
cursor.execute(query)

# Получение и вывод результатов
results = cursor.fetchall()

In [153]:
 # Вывод результатов
print("Опыт работы | Средняя зарплата")
print("------------+------------------")
for row in results:
        print(f"{row[0]:<11} | {row[1]:>16}")

Опыт работы | Средняя зарплата
------------+------------------


In [ ]:
# выводы по предметному анализу


Из 1771 вакансий связанных с данными, только 51 подходит для джунов, из них 357 требуют знания Python.

## Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

На рынке высокий спрос на DS специалистов. Но новичкам надо больше опыта, т.к. вакансий для джнов всего 51. КЛбчевые навыки - SQL  и Python